# The Price is Right

Today we build a more complex solution for estimating prices of goods.

1. Day 2.0 notebook: create a RAG database with our 400,000 training data
2. Day 2.1 notebook: visualize in 2D
3. Day 2.2 notebook: visualize in 3D
4. Day 2.3 notebook: build and test a RAG pipeline with GPT-4o-mini
5. Day 2.4 notebook: (a) bring back our Random Forest pricer (b) Create a Ensemble pricer that allows contributions from all the pricers

Phew! That's a lot to get through in one day!

## PLEASE NOTE:

We already have a very powerful product estimator with our proprietary, fine-tuned LLM. Most people would be very satisfied with that! The main reason we're adding these extra steps is to deepen your expertise with RAG and with Agentic workflows.

## We will go fast today! Hold on to your hat..

In [15]:
# imports

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI, AzureOpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from testing import Tester

In [16]:
# environment

load_dotenv(override=True)
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_api_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT')
if azure_openai_api_key:
    print(f"Azure OpenAI API Key exists and begins {azure_openai_api_key[:8]}")
else:
    print("Azure OpenAI API Key not set") 

Azure OpenAI API Key exists and begins 75Ro1MBh


In [17]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [18]:
# Another import after Logging in to Hugging Face - thank you Trung N.!

from items import Item

In [19]:
openai = AzureOpenAI(
    api_key=azure_openai_api_key,
    api_version="2025-01-01-preview",
    azure_endpoint=azure_openai_api_endpoint
)
OPENAI_MODEL = "gpt-4o"

In [20]:
# Load in the test pickle file
# See the section "Back to the PKL files" in the day2.0 notebook
# for instructions on obtaining this test.pkl file

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [14]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [21]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [22]:
DB = "products_vectorstore"

In [23]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [30]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [31]:
description(test[0])

"OEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K"

In [32]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [33]:
def vector(item):
    return model.encode([description(item)])

In [34]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [35]:
print(test[1].prompt)

How much does this cost to the nearest dollar?

Motorcraft YB3125 Fan Clutch
Motorcraft YB3125 Fan Clutch Package Dimensions 25.146 cms (L) x 20.066 cms (W) x 15.494 cms (H) Package Quantity 1 Product Type Auto Part Country Of Origin China Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch, Weight 5 pounds, Dimensions 10 x 7.63 x 6.25 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Rank Automotive Automotive Replacement Engine Fan Clutches 583, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of the U.S. Learn More, Available October 10, 2007

Price is $225.00


In [37]:
documents, prices = find_similars(test[1])

In [38]:
print(make_context(documents, prices))

To provide some context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Four Seasons 78348 New AC Compressor
New Compressor And Clutch Four Seasons 78348 New Nippondenso 10S17F Compressor w/ Clutch Part number 78348 Package Weight 14.55 pounds Package Dimensions 20.574 H x 28.194 L x 18.034 W (centimeters) Manufacturer Four Seasons, Brand Four Seasons, Weight 6 Pounds, Dimensions 11.3 x 7.2 x 7.8 inches, Country of Origin China, model number 78348, Manufacturer Part 78348, Voltage 120 Volts, Rank Automotive Automotive Replacement Air Conditioning Compressors 1383, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of
Price is $186.99

Potentially related product:
Motorcraft YH-23 Blower Motor Resistor
Motorcraft YH-23 Blower Motor Resistor Package Dimensions 6.82 L x 2.36 H x 4.63 W (inches) Fit type Vehicle Specific Package Weight 1.83 pounds Country 

In [39]:
print(messages_for(test[1], documents, prices))

[{'role': 'system', 'content': 'You estimate prices of items. Reply only with the price, no explanation'}, {'role': 'user', 'content': 'To provide some context, here are some other items that might be similar to the item you need to estimate.\n\nPotentially related product:\nFour Seasons 78348 New AC Compressor\nNew Compressor And Clutch Four Seasons 78348 New Nippondenso 10S17F Compressor w/ Clutch Part number 78348 Package Weight 14.55 pounds Package Dimensions 20.574 H x 28.194 L x 18.034 W (centimeters) Manufacturer Four Seasons, Brand Four Seasons, Weight 6 Pounds, Dimensions 11.3 x 7.2 x 7.8 inches, Country of Origin China, model number 78348, Manufacturer Part 78348, Voltage 120 Volts, Rank Automotive Automotive Replacement Air Conditioning Compressors 1383, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of\nPrice is $186.99\n\nPotentially related product:\nMotorcraft YH-23 Blower Motor Resistor\nMotorcra

In [42]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [43]:
get_price("The price for this is $99.99")

99.99

In [50]:
# The function for gpt-4o-mini

def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model='gpt-4o',
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [51]:
gpt_4o_mini_rag(test[1])

102.49

In [52]:
test[1].price

225.11

In [55]:
Tester.test(gpt_4o_mini_rag, test)

1: Guess: $295.41 Truth: $374.41 Error: $79.00 SLE: 0.06 Item: OEM AC Compressor w/A/C Repair Kit For F...
2: Guess: $102.00 Truth: $225.11 Error: $123.11 SLE: 0.62 Item: Motorcraft YB3125 Fan Clutch
3: Guess: $89.99 Truth: $61.68 Error: $28.31 SLE: 0.14 Item: Dorman 603-159 Front Washer Fluid Reserv...
4: Guess: $399.99 Truth: $599.99 Error: $200.00 SLE: 0.16 Item: HP Premium 17.3-inch HD Plus Touchscreen...
5: Guess: $15.99 Truth: $16.99 Error: $1.00 SLE: 0.00 Item: 5-Position Super Switch Pickup Selector ...
6: Guess: $15.99 Truth: $31.99 Error: $16.00 SLE: 0.44 Item: Horror Bookmarks, Resin Horror Bookmarks...
7: Guess: $119.99 Truth: $101.79 Error: $18.20 SLE: 0.03 Item: SK6241 - Stinger 4 Gauge 6000 Series Pow...
8: Guess: $249.00 Truth: $289.00 Error: $40.00 SLE: 0.02 Item: Godox ML60Bi LED Light Kit, Handheld LED...
9: Guess: $499.99 Truth: $635.86 Error: $135.87 SLE: 0.06 Item: Randall RG75DG3PLUS G3 Plus 100-Watt Com...
10: Guess: $89.99 Truth: $65.99 Error: $24.00 SLE: 0.09 

KeyboardInterrupt: 

## Optional Extra: Trying a DeepSeek API call instead of OpenAI

If you have a DeepSeek API key, we will use it here as an alternative implementation; otherwise skip to the next section..

In [ ]:
# Connect to DeepSeek using the OpenAI client python library

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_via_openai_client = OpenAI(api_key=deepseek_api_key,base_url="https://api.deepseek.com")

In [ ]:
# Added some retry logic here because DeepSeek is very oversubscribed and sometimes fails..

def deepseek_api_rag(item):
    documents, prices = find_similars(item)
    retries = 8
    done = False
    while not done and retries > 0:
        try:
            response = deepseek_via_openai_client.chat.completions.create(
                model="deepseek-chat", 
                messages=messages_for(item, documents, prices),
                seed=42,
                max_tokens=8
            )
            reply = response.choices[0].message.content
            done = True
        except Exception as e:
            print(f"Error: {e}")
            retries -= 1
    return get_price(reply)

In [ ]:
deepseek_api_rag(test[1])

In [ ]:
Tester.test(deepseek_api_rag, test)

## And now to wrap this in an "Agent" class

In [ ]:
from agents.frontier_agent import FrontierAgent

In [ ]:
# Let's print the logs so we can see what's going on

import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
agent = FrontierAgent(collection)

In [ ]:
agent.price("Quadcast HyperX condenser mic for high quality podcasting")

In [ ]:
from agents.specialist_agent import SpecialistAgent

In [ ]:
agent2 = SpecialistAgent()

In [ ]:
agent2.price("Quadcast HyperX condenser mic for high quality podcasting")